# tootltips

In [8]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":0,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    71  100    71    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543


# Imports

In [11]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [12]:
def clean_desc_txt(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t","\\r","\\r\\n"  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [13]:
def setup_es_client():
    es_client = Elasticsearch()

    index_name = "dpe_extract_text"
    index_configurations = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "max_result_window": 5000000,

            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "text_to_analyze": {
                    "type": "text",
                    #"analyzer": "french"      
                },
                "category_index":{
                    'type':'keyword'
                }
            }
        }
    }

    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass
    es_client.indices.create(
        index=index_name,
        body=index_configurations
    )
    return es_client

# fonctions

In [14]:
from generate_dpe_annexes_scripts.utils import strip_accents

def gendata(index_name,id_data,data):
    for id_,vr in zip(id_data,data):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

def generate_instruction_from_list(char_list):
    
    new_char_list=list()
    for char in char_list:
        if isinstance(char,str):
            new_char_list.append(char)
        if isinstance(char,tuple):
            new_char_list.append('('+') OR ('.join(char)+')')
    search_instruction = '('+') AND ('.join(new_char_list) +')'
    return search_instruction
def search_from_search_dict(es_client,search_dict,index_name):
    
    s_list = list()
    for label,char_list in  search_dict.items():

        search_instruction =generate_instruction_from_list(char_list)
        search_body = {
            "query": {
                "query_string": {
                    "query": search_instruction,
                    "default_field": "text_to_analyze"
                },

            },

        }
        #es_client.count(index=index_name, body=search_body)

        a_dict=es_client.search(index=index_name, body=search_body,size=500000)

        hits = a_dict['hits']['hits']

        s=pd.Series(index=[el['_source']['id'] for el in hits])
        s[:]=label
        s_list.append(s)
    s_all=pd.concat(s_list)

    return s_all

def search_and_affect(data_to_search,id_col,val_col,search_dict,index_name='dpe_extract_text',agg_method='keep_first'):
    es_client = setup_es_client()
    bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
    res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

    res_table=res_serie.to_frame('label')

    res_table.index.name = 'id'
    res_table=res_table.reset_index()


# #     grp = df_drop.groupby('id').label.apply(lambda x: ' + '.join(sorted(list(set(x))))).reset_index()
# #     m=data.merge(grp,on='id',how='left')

#     m.label=m.label.fillna('indetermine')
    return res_table
def categorize_search_res(table,label_cat=None,category_dict=None):
    
    if category_dict is not None:
        table['category']=table.label.replace(category_dict)
    
    if label_cat is not None:
        table['label']=pd.Categorical(table.label,categories=label_cat,ordered=True)

    #df_drop = df.sort_values('label').drop_duplicates(subset=['id','category'])
    
    table.label = table.label.fillna('indetermine')
    return table

def agg_concat_label_text(table,col_label='label',col_vr='valeur_renseignee'):
    table[[col_label,col_vr]]=table[[col_label,col_vr]].fillna('indetermine')
    agg = table.groupby('td001_dpe_id').agg({

    col_label:lambda x: ' + '.join(sorted(list(set(x)))),
    col_vr:lambda x: ' + '.join(sorted(list(set(x))))
    })

    for col in agg:
        agg[col] =  agg[col].apply(lambda x:' + '.join(sorted(list(set([el for el in x.split(' + ') if el!='indetermine'])))))
        agg[col] = agg[col].replace('','indetermine')
    return agg

# load tables

In [15]:
from generate_dpe_annexes_scripts.utils import reload_package

In [16]:
import generate_dpe_annexes_scripts.trtvtables
reload_package(generate_dpe_annexes_scripts.trtvtables)

In [17]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths
from generate_dpe_annexes_scripts.trtvtables import DPETrTvTables

trtv = DPETrTvTables()

if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='94':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td002 = pd.read_csv(dept_dir/'td002_consommation.csv',dtype=str)
            td016 = pd.read_csv(dept_dir/'td016_facture.csv',dtype=str)

            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
            td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
            td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)

            td012_p = pd.read_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv', dtype=str)
            td014_p = pd.read_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv', dtype=str)
            td001_sys_ch= pd.read_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv', dtype=str)
            td001_sys_ecs= pd.read_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            td001_sys_ch.columns = [col.replace('_chauffage','_ch') for col in td001_sys_ch]

            print(dept_dir)
            break
            # ENVELOPPE PROCESSING
            td001 =td001.rename(columns={'id':'td001_dpe_id'})

tv025_intermittence_type_ch not found
tv026_classe_inertie_plancher_bas not found
tv026_classe_inertie_plancher_haut not found
tv026_classe_inertie_paroi_verticale not found
tv026_classe_inertie_classe_inertie not found
tv025_intermittence_type_emetteur not found
tv029_simu_type_distribution not found
D:\data\dpe_full\depts\94
D:\data\dpe_full\depts\94


# exploration

In [18]:

td005_mur = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '9']
td005_mur.valeur_renseignee = td005_mur.valeur_renseignee.fillna('indetermine')

td005_mur.valeur_renseignee = td005_mur.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_mur.valeur_renseignee = td005_mur.valeur_renseignee.apply(lambda x:clean_desc_txt(x))



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [29]:
murs_materiau_search_dict = {
 'Murs en pierre de taille et moellons avec remplissage tout venant':'(pierre AND taille) OR (moellons AND remplissage)',
 "Murs en pierre de taille et moellons constitué d'un seul matériaux":'(pierre AND taille) OR (moellons)',
 "Murs en pisé ou béton de terre stabilisé (à partir d'argile crue)":'terre AND stabilise AND beton',
 'Murs en pan de bois sans remplissage tout venant': '(bois) AND ("avec remplissage")',
 'Murs en pan de bois avec remplissage tout venant': '(bois) AND NOT (rondin) AND NOT ("avec remplissage")',
 'Murs bois (rondin)':'bois AND rondin',
 'Murs en briques pleines simples': "(briques OR briques) AND (pleines OR pleine) AND NOT (lame)",
 "Murs en briques pleines doubles avec lame d'air":"(briques OR briques) AND (pleines OR pleine) AND (lame)",
 'Murs en briques creuses':"(briques OR briques) AND (creuse OR creuses)",
 'Murs en blocs de béton pleins':"beton AND (plein OR pleins)",
 'Murs en blocs de béton creux':"beton AND creux",
 'Murs en béton banché':"beton AND banche",
 'Murs en béton de mâchefer':"beton AND machefer",
 'Monomur terre cuite':"terre AND cuite",
 'Béton cellulaire':"beton AND cellulaire",
 'Cloison de plâtre':"platre",
        'Inconnu':'inconnu',
}




In [27]:
is_=td005_mur.valeur_renseignee.str.contains(' bois')
td005_mur.loc[is_,'valeur_renseignee'].to_dict()

{2525: 'mur bois avec remplissage tout venant non isole donnant sur l exterieur<br  surface : 0 m²  donnant sur : l exterieur  u : 1 7 w m² c  b : 1',
 7108: 'mur bois avec remplissage tout venant donnant sur l exterieur avec isolation interieure  realisee entre 2001 et 2005)<br  surface : 72 m²  donnant sur : l exterieur  u : 0 4 w m² c  b : 1',
 9842: 'pans de bois donnant sur l exterieur avec isolation interieure  10 cm)<br  surface : 0 m²  donnant sur : l exterieur  u : 0.4 w m² c  b : 1',
 12596: ' - mur n 1 mur en pierre de taille moellons ep 50cm non isole     - mur n 2 mur en pans de bois avec remplissage ep <=8cm non isole   ',
 13975: 'pans de bois non isole donnant sur des circulations communes sans ouverture directe sur l exterieur<br  surface : 10 m²  donnant sur : des circulations communes sans ouverture directe sur l exterieur  u : 2 w m² c  b : 0.8',
 14823: 'pans de bois non isole donnant sur l exterieur<br  surface : 60 m²  donnant sur : l exterieur  u : 2 w m² c  b :

In [8]:
toto="""
systeme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 
"""
toto="""
 - collectif : solaire avec appoint chaudiere a condensation gaz naturel"""

In [9]:
index_name = 'dpe_extract_text'

In [10]:
search_instruction = '((reseau de chaleur) OR (reseaux de chaleurs) OR (reseaux de chaleur))'
search_instruction = generate_instruction_from_list(gen_ecs_search_dict['ecs thermodynamique electrique(PAC ou ballon)'])
search_instruction = generate_instruction_from_list(gen_ecs_search_dict['abscence ecs solaire'])

NameError: name 'gen_ecs_search_dict' is not defined

In [208]:
search_instruction = generate_instruction_from_list(gen_ecs_search_dict['ecs thermodynamique electrique(PAC ou ballon)'])


In [209]:
search_instruction

'((pompe AND chaleur) OR (pac) OR (thermodynamique) OR ("air extrait") OR ("air exterieur") OR ("air ambiant"))'

In [214]:
search_instruction ='(("ecs solaire : non") OR ("sans solaire"))'

In [215]:
#search_instruction = 'energie'

In [216]:
import time

In [217]:
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client = setup_es_client()
bulk(es_client, gendata(index_name,['id'],[toto]))

time.sleep(1)
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [218]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [60]:
bulk(es_client, gendata(index_name,['id'],[toto]))


(1, [])

In [53]:
es_client

<Elasticsearch([{}])>

In [54]:
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8630463,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'MeKchHYBLPwkNvcNz20A',
    '_score': 0.8630463,
    '_source': {'id': 'id',
     'text_to_analyze': '\nsysteme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 \n'}}]}}

In [ ]:

s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

In [46]:
m_ch = m

In [81]:
id_=m_ch.vr_source_ft.str.contains('air')&m_ch.vr_source_ft.str.contains('eau')&m_ch.vr_source_ft.str.contains('pac')

In [82]:
m_ch.loc[id_].to_dict(orient='records')

[{'td001_dpe_id': '1009801',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split multisplit  energie : electrique)  avec equipement d intermittence central sans minimum de temperature  absence de regulation par piece  reseau de distribution isole',
  'type_installation_ch_ft': 'indetermine',
  'energie_ch_ft': 'Electricité non renouvelable',
  'gen_ch_lib_desc': 'pac air/air',
  'vr_source_desc': 'installation de chauffage \r - chauffage 1  pac air air split multisplit  energie electrique  reseau individuel',
  'type_installation_ch_desc': 'Chauffage Individuel',
  'energie_ch_desc': 'Chauffage Individuel',
  'gen_ch_lib_infer_concat': nan,
  'mix_energetique_ch': nan,
  'type_installation_ch_concat': nan,
  'nom_methode_dpe_norm': 'FACTURE'},
 {'td001_dpe_id': '1009804',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split m

In [76]:
m_ch.loc[m_ch.gen_ch_lib_ft.fillna('na').str.contains('air/eau')]

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm


In [77]:
m_ch.gen_ch_lib_<Q<

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm
0,1000015,convecteurs electriques nfc,convecteur electrique nfc :\r - type de produ...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteur electrique nfc\r,indetermine,indetermine,convecteurs electriques nfc,Electricité non renouvelable,Chauffage Individuel,3CL 2012
1,1000040,convecteurs electriques nfc,convecteurs electriques nfc systeme individue...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteurs electriques nfc systeme individuel),Chauffage Individuel,Chauffage Individuel,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
2,1000043,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1991,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
3,1000076,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur<br ...,indetermine,indetermine,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur,indetermine,indetermine,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
4,1000112,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145974,9402027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145975,9410001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145976,9421242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3CL 2012
145977,9421904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE


In [61]:
m_ch.gen_ch_lib_ft.value_counts().to_dict()

{'reseau de chaleur': 20592,
 'chaudiere gaz indetermine': 18914,
 'indetermine': 11751,
 'chaudiere gaz basse temperature': 10978,
 'chaudiere gaz standard': 10088,
 'convecteurs electriques nfc': 9708,
 'chaudiere gaz condensation': 8251,
 'autres emetteurs a effet joule': 8158,
 'radiateurs electriques': 6936,
 'panneaux rayonnants electriques nfc': 4599,
 'chaudiere fioul indetermine': 3706,
 'chaudiere energie indetermine indetermine': 2772,
 'chaudiere fioul standard': 1348,
 'pac air/air': 1155,
 'chaudiere energie indetermine basse temperature': 647,
 'chaudiere energie indetermine condensation': 454,
 'chaudiere fioul basse temperature': 441,
 'pac eau/eau': 397,
 'plafonds/planchers rayonnants electriques nfc': 386,
 'convecteurs electriques nfc + panneaux rayonnants electriques nfc': 246,
 'poele ou insert bois': 230,
 'convecteurs electriques nfc + radiateurs electriques': 175,
 'chaudiere bois': 120,
 'pac geothermique': 116,
 'panneaux rayonnants electriques nfc + radiate

In [53]:
s=m_ch.loc[m_ch.nom_methode_dpe_norm=='FACTURE'].gen_ch_lib_ft.fillna('indetermine').value_counts()

In [57]:
s.loc['indetermine']/s.sum()

0.15338467987006182

In [55]:
(s=='indetermine').mean()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0.0

In [35]:
m_ch.groupby('nom_methode_dpe_norm').gen_ch_lib_ft.value_counts()

nom_methode_dpe_norm  gen_ch_lib_ft                                                              
3CL 2005              convecteurs electriques nfc                                                    45
                      radiateurs electriques                                                         15
                      panneaux rayonnants electriques nfc                                            13
                      chaudiere gaz basse temperature                                                12
                      chaudiere gaz standard                                                          9
                                                                                                     ..
FACTURE               chaudiere energie indetermine indetermine + convecteurs electriques nfc         1
                      chaudiere gaz basse temperature + chaudiere gaz indetermine                     1
                      chaudiere gaz indetermine + indetermine         

In [ ]:
m.type_installation_ch_ft.value_counts()

In [ ]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')